<a href="https://colab.research.google.com/github/leonardo3108/robustez-query/blob/main/code/calculate_metric_base_trec2020_full.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparação

## Carga dos pacotes pygaggle e pyserini

In [1]:
!pip install git+https://github.com/castorini/pygaggle.git

  Cloning https://github.com/castorini/pygaggle.git to /tmp/pip-req-build-dc96ny72
  Running command git clone -q https://github.com/castorini/pygaggle.git /tmp/pip-req-build-dc96ny72
  Running command git submodule update --init --recursive -q
     |████████████████████████████████| 43 kB 1.4 MB/s 
     |████████████████████████████████| 9.1 MB 10.0 MB/s 
     |████████████████████████████████| 75.5 MB 6.3 kB/s 
     |████████████████████████████████| 22.3 MB 39.5 MB/s 
     |████████████████████████████████| 25.9 MB 1.5 MB/s 
     |████████████████████████████████| 12.8 MB 35.4 MB/s 
     |████████████████████████████████| 6.0 MB 45.4 MB/s 
     |████████████████████████████████| 454.3 MB 16 kB/s 
     |████████████████████████████████| 3.3 MB 30.7 MB/s 
     |████████████████████████████████| 72 kB 984 kB/s 
     |████████████████████████████████| 2.2 MB 33.6 MB/s 
     |████████████████████████████████| 1.2 MB 40.1 MB/s 
     |████████████████████████████████| 85 kB 4.6 MB/s 
     

In [2]:
!pip install pyserini

## Carga do pacote Faiss (GPU)

In [3]:
!pip install faiss-gpu

     |████████████████████████████████| 89.7 MB 12 kB/s 


## Carga do dataset MsMarco Passage

In [4]:
from pyserini.search import SimpleSearcher

# `SimpleSearcher` defaults to BM25 scoring function.
searcher = SimpleSearcher.from_prebuilt_index('msmarco-passage')

index-msmarco-passage-20201117-f87c94.tar.gz: 0.00B [00:00, ?B/s]

Attempting to initialize pre-built index msmarco-passage.


index-msmarco-passage-20201117-f87c94.tar.gz: 2.07GB [01:01, 35.8MB/s]                            


Extracting /root/.cache/pyserini/indexes/index-msmarco-passage-20201117-f87c94.tar.gz into /root/.cache/pyserini/indexes/index-msmarco-passage-20201117-f87c94.1efad4f1ae6a77e235042eff4be1612d...
Initializing msmarco-passage...


## Utility library

In [5]:
!wget -nc https://raw.githubusercontent.com/leonardo3108/robustez-query/main/code/local/util_bd_dataframe.py

--2021-11-23 00:13:24--  https://raw.githubusercontent.com/leonardo3108/robustez-query/main/code/local/util_bd_dataframe.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11889 (12K) [text/plain]
Saving to: ‘util_bd_dataframe.py’

util_bd_dataframe.p 100%[===================>]  11.61K  --.-KB/s    in 0s      

2021-11-23 00:13:24 (59.0 MB/s) - ‘util_bd_dataframe.py’ saved [11889/11889]



In [12]:
!wget --directory-prefix=/content/data/ -nc https://raw.githubusercontent.com/leonardo3108/robustez-query/main/data/tab_original_query.csv

--2021-11-23 00:17:36--  https://raw.githubusercontent.com/leonardo3108/robustez-query/main/data/tab_original_query.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3302 (3.2K) [text/plain]
Saving to: ‘/content/data/tab_original_query.csv’

tab_original_query. 100%[===================>]   3.22K  --.-KB/s    in 0s      

2021-11-23 00:17:36 (34.0 MB/s) - ‘/content/data/tab_original_query.csv’ saved [3302/3302]



In [25]:
! wget -nc https://raw.githubusercontent.com/leonardo3108/robustez-query/main/data/tab_calculated_metric.csv -P data

--2021-11-23 00:42:48--  https://raw.githubusercontent.com/leonardo3108/robustez-query/main/data/tab_calculated_metric.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 113430 (111K) [text/plain]
Saving to: ‘data/tab_calculated_metric.csv’

tab_calculated_metr 100%[===================>] 110.77K  --.-KB/s    in 0.01s   

2021-11-23 00:42:48 (7.26 MB/s) - ‘data/tab_calculated_metric.csv’ saved [113430/113430]



In [14]:
! wget -nc https://raw.githubusercontent.com/leonardo3108/robustez-query/main/data/tab_noise_kind.csv -P data
! wget -nc https://raw.githubusercontent.com/leonardo3108/robustez-query/main/data/tab_search_context.csv -P data

--2021-11-23 00:20:52--  https://raw.githubusercontent.com/leonardo3108/robustez-query/main/data/tab_noise_kind.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 382 [text/plain]
Saving to: ‘data/tab_noise_kind.csv’

tab_noise_kind.csv  100%[===================>]     382  --.-KB/s    in 0s      

2021-11-23 00:20:52 (16.0 MB/s) - ‘data/tab_noise_kind.csv’ saved [382/382]

--2021-11-23 00:20:52--  https://raw.githubusercontent.com/leonardo3108/robustez-query/main/data/tab_search_context.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK

In [18]:
! wget -nc https://raw.githubusercontent.com/leonardo3108/robustez-query/main/data/tab_noisy_query.csv -P data

--2021-11-23 00:24:53--  https://raw.githubusercontent.com/leonardo3108/robustez-query/main/data/tab_noisy_query.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20537 (20K) [text/plain]
Saving to: ‘data/tab_noisy_query.csv’

tab_noisy_query.csv 100%[===================>]  20.06K  --.-KB/s    in 0s      

2021-11-23 00:24:53 (115 MB/s) - ‘data/tab_noisy_query.csv’ saved [20537/20537]



In [16]:
! wget -nc https://raw.githubusercontent.com/leonardo3108/robustez-query/main/data/originals/2020qrels-pass.txt -P data/originals

--2021-11-23 00:21:58--  https://raw.githubusercontent.com/leonardo3108/robustez-query/main/data/originals/2020qrels-pass.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 218617 (213K) [text/plain]
Saving to: ‘data/originals/2020qrels-pass.txt’

2020qrels-pass.txt  100%[===================>] 213.49K  --.-KB/s    in 0.01s   

2021-11-23 00:21:58 (14.0 MB/s) - ‘data/originals/2020qrels-pass.txt’ saved [218617/218617]



In [11]:
import util_bd_dataframe as util_bd_pandas

## Carga do Reranker MonoT5

In [20]:
!nvidia-smi

Tue Nov 23 00:25:13 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    28W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [21]:
from pygaggle.rerank.transformer import MonoT5

reranker = MonoT5() 

Downloading:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.09k [00:00<?, ?B/s]

# Execução

In [22]:
import json
from pygaggle.rerank.base import Query, Text
from pygaggle.rerank.base import hits_to_texts
import math

## Cálculo do BM25

In [27]:
  _, dict_val_idcg10 = util_bd_pandas.read_df_original_query()
  df_noisy_query = util_bd_pandas.read_df_noisy_query()
  dict_judment, _ = util_bd_pandas.load_judment()

  # Calculate dcg10 and ndcg10 of noisy queries
  for noise_kind in df_noisy_query['cod_noise_kind'].unique():
      calculated_metric = {'cod_original_query':[],'dcg10':[],'ndcg10':[], 'qtd_judment_assumed_zero_relevance':[]}
      for index, row in df_noisy_query[df_noisy_query['cod_noise_kind']==noise_kind].iterrows():
          cod_original_query = row["cod_original_query"]
          idcg10 = dict_val_idcg10[cod_original_query]
          query_text = row["text"]

          # calculating dcg10 for query
          dcg10 = 0
          qtd_judment_assumed_zero_relevance = 0

          query_busca = Query(query_text)
          hits = searcher.search(query_busca.text, k=10)
          doctos_returned = hits_to_texts(hits)

          for i, doc in enumerate(doctos_returned):
              if i >= 10:
                  raise 'Error: more than 10 retrieved!'
              docid = doc.metadata["docid"]

              if (cod_original_query, docid) not in dict_judment:
                  print(f'Query and passage_id not in judment:{(cod_original_query, docid)}. Assumed zero relevance. ')
                  qtd_judment_assumed_zero_relevance += 1
              eval = dict_judment.get((cod_original_query, docid), 0)

              dcg10 += (2**int(eval)-1) / math.log2(i+2)
              # calculate ndcg10 for query
          ndcg10 = dcg10 / idcg10
          # print(f"query {cod_original_query} has dcg10 {dcg10} ndcg10 {ndcg10} for idcg10 {idcg10} ")


          calculated_metric['cod_original_query'].append(cod_original_query)
          calculated_metric['dcg10'].append(dcg10)
          calculated_metric['ndcg10'].append(ndcg10)
          calculated_metric['qtd_judment_assumed_zero_relevance'].append(qtd_judment_assumed_zero_relevance)
      # print(calculated_metric)
      util_bd_pandas.save_calculated_metric(calculated_metric, cod_search_context=3, cod_noise_kind=noise_kind)    




Loaded len: 11386 judments; judment[0] ((23849, '1020327'), 2, 'highly relevant')
Query and passage_id not in judment:(1121353, '8083913'). Assumed zero relevance. 
Query and passage_id not in judment:(1121353, '5300162'). Assumed zero relevance. 
Query and passage_id not in judment:(1121353, '3541849'). Assumed zero relevance. 
saving calculated metric cod_search_context?3, cod_noise_kind:0
Query and passage_id not in judment:(1121353, '8083913'). Assumed zero relevance. 
Query and passage_id not in judment:(1121353, '5300162'). Assumed zero relevance. 
Query and passage_id not in judment:(1121353, '3541849'). Assumed zero relevance. 
saving calculated metric cod_search_context?3, cod_noise_kind:1
Query and passage_id not in judment:(1121353, '8083913'). Assumed zero relevance. 
Query and passage_id not in judment:(1121353, '5300162'). Assumed zero relevance. 
Query and passage_id not in judment:(1121353, '3541849'). Assumed zero relevance. 
saving calculated metric cod_search_contex

## Rank BM25 (1000 melhores ranks)

In [30]:
import json
from pygaggle.rerank.base import Query, Text
from pygaggle.rerank.base import hits_to_texts

In [34]:
  _, dict_val_idcg10 = util_bd_pandas.read_df_original_query()
  df_noisy_query = util_bd_pandas.read_df_noisy_query()
  dict_judment, _ = util_bd_pandas.load_judment()

  # Calculate dcg10 and ndcg10 of noisy queries
  for noise_kind in df_noisy_query['cod_noise_kind'].unique():
      calculated_metric = {'cod_original_query':[],'dcg10':[],'ndcg10':[], 'qtd_judment_assumed_zero_relevance':[]}
      for index, row in df_noisy_query[df_noisy_query['cod_noise_kind']==noise_kind].iterrows():
          cod_original_query = row["cod_original_query"]
          idcg10 = dict_val_idcg10[cod_original_query]
          query_text = row["text"]

          # calculating dcg10 for query
          dcg10 = 0
          qtd_judment_assumed_zero_relevance = 0

          query_busca = Query(query_text)
          hits = searcher.search(query_busca.text, k=1000)

          doctos_returned_bm25 = hits_to_texts(hits)

          doctos_returned = reranker.rerank(query_busca, doctos_returned_bm25)

          for i, doc in enumerate(doctos_returned[:10]):
              if i >= 10:
                  raise 'Error: more than 10 retrieved!'
              docid = doc.metadata["docid"]

              if (cod_original_query, docid) not in dict_judment:
                  print(f'Query and passage_id not in judment:{(cod_original_query, docid)}. Assumed zero relevance. ')
                  qtd_judment_assumed_zero_relevance += 1
              eval = dict_judment.get((cod_original_query, docid), 0)

              dcg10 += (2**int(eval)-1) / math.log2(i+2)
              # calculate ndcg10 for query
          ndcg10 = dcg10 / idcg10
          # print(f"query {cod_original_query} has dcg10 {dcg10} ndcg10 {ndcg10} for idcg10 {idcg10} ")


          calculated_metric['cod_original_query'].append(cod_original_query)
          calculated_metric['dcg10'].append(dcg10)
          calculated_metric['ndcg10'].append(ndcg10)
          calculated_metric['qtd_judment_assumed_zero_relevance'].append(qtd_judment_assumed_zero_relevance)
      util_bd_pandas.save_calculated_metric(calculated_metric, cod_search_context=1, cod_noise_kind=noise_kind)    
      print(f" Done for noise_kind: {noise_kind}")



Loaded len: 11386 judments; judment[0] ((23849, '1020327'), 2, 'highly relevant')
Query and passage_id not in judment:(324585, '1514877'). Assumed zero relevance. 
Query and passage_id not in judment:(336901, '2261724'). Assumed zero relevance. 
Query and passage_id not in judment:(405163, '3340708'). Assumed zero relevance. 
Query and passage_id not in judment:(1108651, '8643791'). Assumed zero relevance. 
Query and passage_id not in judment:(1110678, '5272210'). Assumed zero relevance. 
Query and passage_id not in judment:(1115210, '7238888'). Assumed zero relevance. 
Query and passage_id not in judment:(1121353, '8684415'). Assumed zero relevance. 
Query and passage_id not in judment:(1121353, '643775'). Assumed zero relevance. 
Query and passage_id not in judment:(1121353, '2213823'). Assumed zero relevance. 
saving calculated metric cod_search_context?1, cod_noise_kind:0
 Done for noise_kind: 0
Query and passage_id not in judment:(23849, '4806514'). Assumed zero relevance. 
Query

In [33]:
from google.colab import files


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [35]:
files.download( "data/tab_calculated_metric.csv" ) 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>